In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
import seaborn as sns
import pandas as pd
from scipy.stats import norm

from ipywidgets import interact
%matplotlib qt
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui
plt.rcdefaults()

# figure styling
font=18
font_axis=20
plt.rcParams.update({'font.size': font})  # Set the desired font size
# plt.rcParams['text.usetex'] = True
plt.rcParams['figure.facecolor'] = 'white'  # Background color for figures
plt.rcParams['axes.facecolor'] = 'white'    # Background color for axes
# Set the font to Times New Roman globally
# plt.rcParams["font.family"] = "Times New Roman"
# plt.rcParams["mathtext.fontset"] = "stix"  # Use a math font for mathtext (like Greek symbols)

# Generate synthetic data
np.random.seed(42)
n_groups = 100       # Number of groups
n_per_group = 7      # Observations per group
group_idx = np.repeat(np.arange(n_groups), n_per_group)


data = np.load('data.npy')

## Generate synthetic data
# group_means = np.random.uniform(1, 10, size=n_groups)
# group_slopes = np.random.uniform(5, 7, size=n_groups)
# data = []
# for group in range(n_groups):
#     x = np.linspace(0, 10, n_per_group)
#     # y = group_means[group] + group_slopes[group] * x + np.random.normal(0, 1, size=n_per_group)
#     y = group_slopes[group] * x + np.random.normal(0, 1, size=n_per_group)
#     data.append((x, y))

# Flatten data for modeling
x = np.concatenate([d[0] for d in data])
y = np.concatenate([d[1] for d in data])


plt.figure(figsize=(5, 5))
plt.scatter(x, y)  # Create a scatter plot for each x, y pair
plt.xlim(0, np.max(x)+np.max(x)*0.1)
plt.ylim(0, np.max(y)+np.max(y)*0.1)
plt.xlabel('x values')
plt.ylabel('y values')
# plt.title('Scatter Plot of x and y')
plt.show()
plt.tight_layout()

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# Hierarchical Bayesian Model for strain-tip displacement data
with pm.Model() as model:
    # Hyperpriors for group means and slopes
    mu_beta = pm.Normal("mu_beta", mu=0, sigma=10)
    sigma_beta = pm.HalfNormal("sigma_beta", sigma=10)
    
    # Group-level priors
    beta = pm.Normal("beta", mu=mu_beta, sigma=sigma_beta, shape=n_groups)
    
    # Residual standard deviation
    sigma = pm.HalfNormal("sigma", sigma=10)

    # Likelihood
    y_obs = pm.Normal(
        "y_obs",
        mu=beta[group_idx] * x,
        sigma=sigma,
        observed=y,
    )
    
    # MCMC sampling
    trace = pm.sample(1000, return_inferencedata=True)

# Posterior Predictive Check
with model:
    posterior_pred = pm.sample_posterior_predictive(trace)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_beta, sigma_beta, beta, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 36 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Sampling: [y_obs]


Output()

In [4]:
# Convert posterior samples into a pandas DataFrame
posterior_samples = {
    # "mu_alpha": trace.posterior["mu_alpha"].values.flatten(),
    # "sigma_alpha": trace.posterior["sigma_alpha"].values.flatten(),
    r'$\mu_{\theta(1)}$': trace.posterior["mu_beta"].values.flatten(),
    r'$\Sigma_{\theta(1)}$': trace.posterior["sigma_beta"].values.flatten(),
    r'$\sigma$': trace.posterior["sigma"].values.flatten(),
}

df = pd.DataFrame(posterior_samples)

# Create the PairGrid
g = sns.PairGrid(df)
g.map_lower(sns.kdeplot, fill=True, cmap="Blues")  # KDE on the lower side
g.map_diag(sns.histplot, kde=True, color="blue")   # Histogram on the diagonal
g.map_upper(sns.scatterplot, alpha=0.5, color="darkblue")  # Scatter above the diagonal

plt.gcf().text(0.03, 0.03, "(a)", fontsize=20, font='Times New Roman', verticalalignment='bottom', horizontalalignment='left')
# Add titles and adjust layout
# g.figure.suptitle("Pair Plot: Joint Posterior with Scatter Above Diagonal")
plt.show()
plt.tight_layout()

In [5]:
# Assuming you already have the posterior samples as 'trace'

# Get posterior samples for the parameters
# mu_alpha_samples = trace.posterior["mu_alpha"].values.flatten()
# sigma_alpha_samples = trace.posterior["sigma_alpha"].values.flatten()
mu_beta_samples = trace.posterior["mu_beta"].values.flatten()
sigma_beta_samples = trace.posterior["sigma_beta"].values.flatten()
sigma_samples = trace.posterior["sigma"].values.flatten()

# Make predictions based on these posterior samples
# Here we assume a simple linear model for prediction: y = mu_alpha + mu_beta * x + sigma
# Replace 'x_data' with your actual predictor values (e.g., the independent variable)

x_data = np.linspace(0, 15, 100)  # Example predictor values (replace with your actual data)

# Create a container for the predictions
predictions = []

for i in range(len(mu_beta_samples)):
    # Generate prediction for each sample of the parameters
    # y_pred = mu_alpha_samples[i] + mu_beta_samples[i] * x_data + np.random.normal(0, sigma_samples[i], len(x_data))
    y_pred = (sigma_beta_samples[i] + mu_beta_samples[i]) * x_data + np.random.normal(0, sigma_samples[i], len(x_data))
    predictions.append(y_pred)

# Convert predictions to a numpy array
predictions = np.array(predictions)

# Calculate the 95% and 99% confidence intervals (credible intervals)
lower_bound_95 = np.percentile(predictions, 5, axis=0)  # 5th percentile for lower bound (90% CI)
upper_bound_95 = np.percentile(predictions, 95, axis=0)  # 95th percentile for upper bound (90% CI)

lower_bound_99 = np.percentile(predictions, 0.5, axis=0)  # 0.5th percentile for lower bound (99% CI)
upper_bound_99 = np.percentile(predictions, 99.5, axis=0)  # 99.5th percentile for upper bound (99% CI)

# Data from analytical caluclation of strain and tip displacement for dogbone UFT specimen (ref: )
locX=[0, 16.5]
locY=[0, 2026.25]

# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 5))
ax.plot(x_data, np.mean(predictions, axis=0), color='blue', label='Posterior Predictive Mean')
ax.plot(locX, locY, linestyle='--', color='red', label='Analytical estimation')
ax.fill_between(x_data, lower_bound_99, upper_bound_99, color='blue', alpha=0.2, label="99% Confidence Interval")
ax.fill_between(x_data, lower_bound_95, upper_bound_95, color='orange', alpha=0.2, label="90% Confidence Interval")
ax.scatter(x, y, color='black', s=10)
ax.set_xlabel('Tip displacement ($\mu$m)')
ax.set_ylabel('Strain ($\mu$)')
plt.xlim(0, np.max(x)+np.max(x)*0.1)
plt.ylim(0, 2000)
ax.legend(frameon=False,fontsize=14)
fig.text(0.07, 0.07, "(b)", fontsize=20, font='Times New Roman', verticalalignment='bottom', horizontalalignment='left')
plt.tight_layout()
plt.show()


<>:46: SyntaxWarning: invalid escape sequence '\m'
<>:47: SyntaxWarning: invalid escape sequence '\m'
<>:46: SyntaxWarning: invalid escape sequence '\m'
<>:47: SyntaxWarning: invalid escape sequence '\m'
C:\Users\lt24550\AppData\Local\Temp\ipykernel_22664\2136198639.py:46: SyntaxWarning: invalid escape sequence '\m'
  ax.set_xlabel('Tip displacement ($\mu$m)')
C:\Users\lt24550\AppData\Local\Temp\ipykernel_22664\2136198639.py:47: SyntaxWarning: invalid escape sequence '\m'
  ax.set_ylabel('Strain ($\mu$)')
